In [19]:
import subprocess
import pandas as pd
import fasttext

In [20]:
retail_data = pd.read_csv("train.csv")
retail_data.head()

,ImgId,title,description,categories
0,B000HYL1V6,TUNGSTEN SOLDER PICK WITH HANDLE,Solder Pick for picking up molten solder when ...,"Arts, Crafts & Sewing"
1,B00006HXWY,Write Right 98167 Screen Protector for Sony T615C,We all screen. And we all need to protect thos...,Cell Phones & Accessories
2,B000GAWSBS,Casio Mens DBC310-1 Databank 300 Digital Watch...,"Bringing you precision at a glance, the Casio ...","Clothing, Shoes & Jewelry"
3,B000040JOL,Factory-Reconditioned DEWALT DW260KR Heavy-Dut...,Factory-Reconditioned DEWALT DW260KR Heavy-Dut...,Tools & Home Improvement
4,B00006IB78,Energizer 2 in 1 Light,This twoway light features a bright flashlight...,Health & Personal Care


In [21]:
# Delete the ImdId column
img_id_col = retail_data.columns[0]
retail_data_processed = retail_data.drop([img_id_col], axis=1)
retail_data_processed.head()

,title,description,categories
0,TUNGSTEN SOLDER PICK WITH HANDLE,Solder Pick for picking up molten solder when ...,"Arts, Crafts & Sewing"
1,Write Right 98167 Screen Protector for Sony T615C,We all screen. And we all need to protect thos...,Cell Phones & Accessories
2,Casio Mens DBC310-1 Databank 300 Digital Watch...,"Bringing you precision at a glance, the Casio ...","Clothing, Shoes & Jewelry"
3,Factory-Reconditioned DEWALT DW260KR Heavy-Dut...,Factory-Reconditioned DEWALT DW260KR Heavy-Dut...,Tools & Home Improvement
4,Energizer 2 in 1 Light,This twoway light features a bright flashlight...,Health & Personal Care


In [22]:
# Make the categories column the first column
cols = retail_data_processed.columns.tolist()
cols = cols[-1:] + cols[:-1]
retail_data_processed = retail_data_processed[cols]
retail_data_processed.head()

,categories,title,description
0,"Arts, Crafts & Sewing",TUNGSTEN SOLDER PICK WITH HANDLE,Solder Pick for picking up molten solder when ...
1,Cell Phones & Accessories,Write Right 98167 Screen Protector for Sony T615C,We all screen. And we all need to protect thos...
2,"Clothing, Shoes & Jewelry",Casio Mens DBC310-1 Databank 300 Digital Watch...,"Bringing you precision at a glance, the Casio ..."
3,Tools & Home Improvement,Factory-Reconditioned DEWALT DW260KR Heavy-Dut...,Factory-Reconditioned DEWALT DW260KR Heavy-Dut...
4,Health & Personal Care,Energizer 2 in 1 Light,This twoway light features a bright flashlight...


In [23]:
# Replace the spaces in the first column with underscores
retail_data_processed["categories"] = retail_data_processed["categories"].str.replace(" ", "-")
retail_data_processed["categories"] = retail_data_processed["categories"].str.replace(",", "")
retail_data_processed["categories"] 

0             Arts-Crafts-&-Sewing
1        Cell-Phones-&-Accessories
2         Clothing-Shoes-&-Jewelry
3         Tools-&-Home-Improvement
4           Health-&-Personal-Care
                   ...            
46224            Sports-&-Outdoors
46225         Arts-Crafts-&-Sewing
46226         Arts-Crafts-&-Sewing
46227       Grocery-&-Gourmet-Food
46228         Arts-Crafts-&-Sewing
Name: categories, Length: 46229, dtype: object

In [24]:
# Save the processed .csv file
retail_data_processed.to_csv("train.processed.csv", index=False)

In [25]:
# Normalizing the text
process = subprocess.run("""
    cat /home/agastya/Programming/retail/train.processed.csv | \
    tr '[:upper:]' '[:lower:]' | \
    sed -e 's/^/__label__/g' | \
    sed \
        -e "s/'/ ' /g" -e 's/"//g' \
        -e 's/\./ \. /g' \
        -e 's/<br \/>/ /g' \
        -e 's/,/ , /g' \
        -e 's/(/ ( /g' \
        -e 's/)/ ) /g' \
        -e 's/\!/ \! /g' \
        -e 's/\?/ \? /g' \
        -e 's/\;/ /g' \
        -e 's/\:/ /g' | tr -s " " | \
    perl -MList::Util=shuffle \
        -e 'print shuffle(<>);' "$@";
""", capture_output=True, shell=True)

output = process.stdout.decode("utf-8")

In [26]:
# Save the output to a file
with open("retail.train", "w") as training_file:
    training_file.write(output)

In [27]:
# Create the fasttext model and save it
model = fasttext.train_supervised(input="retail.train")
model.save_model("retail.bin")

Read 4M words

Read 4M words
Number of words:  151371
Number of labels: 1991
Progress:   0.8% words/sec/thread:  119466 lr:  0.099223 avg.loss: 17.261974 ETA:   0h 0m12s

Progress:   2.3% words/sec/thread:  118581 lr:  0.097696 avg.loss:  9.173843 ETA:   0h 0m12s

Progress:   5.4% words/sec/thread:  118462 lr:  0.094636 avg.loss:  6.620281 ETA:   0h 0m12s

Progress:   7.8% words/sec/thread:  121102 lr:  0.092169 avg.loss:  5.843977 ETA:   0h 0m11s

...

Progress:  92.5% words/sec/thread:  129942 lr:  0.007518 avg.loss:  3.057617 ETA:   0h 0m 0s

Progress:  95.1% words/sec/thread:  130024 lr:  0.004872 avg.loss:  3.032368 ETA:   0h 0m 0s

Progress:  97.6% words/sec/thread:  130012 lr:  0.002361 avg.loss:  3.012515 ETA:   0h 0m 0s

Progress: 

Progress: 100.0% words/sec/thread:  129810 lr:  0.000000 avg.loss:  2.990862 ETA:   0h 0m 0s


In [28]:
model.predict('TV')

(('__label__',), array([0.83487737]))